In [1]:
# Installing and importing required libraries
!pip install PyPDF2
import PyPDF2
import re


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Defining a function to process PDFs and extract topic outlines and learning outcomes
def process_pdf(input_pdf_path, output_txt_path):
    pdfFileObj = open(input_pdf_path, 'rb')
    pdfReader = PyPDF2.PdfReader(pdfFileObj)

    print(f"Processing {input_pdf_path}")
    print("Total number of pages:", len(pdfReader.pages))
    
# Initializing dictionaries to store extracted content
    content = dict()
    topic = ""
    topic_dict = dict()

    # Iterating through each page of the PDF and extracting text from the current page and split into lines
    for page_num in range(len(pdfReader.pages)):
        t = pdfReader.pages[page_num].extract_text().split('\n')
        line_num = 0

        # Skipping header lines containing 'topic outlines'
        while line_num < len(t):
            if line_num == 0:
                if 'topic outlines' in t[line_num].strip().lower():
                    line_num += 1
                topic_new = re.sub(r'[^A-Za-z ]+', '', t[line_num]).strip()
            
                # Checking if the topic already exists in the content dictionary
                all_keys = [x.lower().strip().replace(" ", "") for x in content.keys()]
                if topic_new.lower().strip().replace(" ", "") in all_keys:
                    topic_new = list(filter(lambda x: x.lower().strip().replace(" ", "") == topic_new.lower().strip().replace(" ", ""), content.keys()))[0]

                # Updating the topic if it has changed
                if topic == topic_new:
                    pass
                else:
                    subtopic = ""
                    subtopic_dict = []
                    topic = topic_new
            topic_dict = content.get(topic, dict())

            # Identifying subtopics i.e. Learning outcoomes for Topics
            subtopic_loc = t[line_num].find("The candidate should be able to:")
            if subtopic_loc != -1:
                subtopic = t[line_num - 1] if subtopic_loc == 0 else t[line_num][:subtopic_loc + 1]
                subtopic_dict = topic_dict.get(subtopic, [])
                tab_loc = t[line_num].find("\t")

                # Extract learning outcomes and appending to the subtopic dictionary
                append_list = t[line_num][tab_loc + 1:] + t[line_num + 1]
                if append_list.find("\t") == -1:
                    subtopic_dict.append(append_list)

                    line_num += 2
                if line_num >= len(t):
                    break
            # Handeling corcer cases, extract learning outcomes from lines with tabs
            tab_loc = t[line_num].find("\t")

            if tab_loc != -1:
                subtopic_dict.append(t[line_num][tab_loc + 1:])

            # Updating the topic dictionary
            topic_dict[subtopic] = subtopic_dict
            content[topic] = topic_dict

            line_num += 1
    
    # Writing the extracted data to an output text files
    with open(output_txt_path, 'w', encoding='utf-8') as output_file:
        for topic, subtopics in content.items():
            output_file.write(f"\nTopic: {topic}\n")
            output_file.write("\t\n\tLearning Outcomes: \n")
            output_file.write("\t\t(For the below Learning Outcomes, The candidate should be able to: )\n")
            for subtopic, learning_outcomes in subtopics.items():
                if subtopic == '':
                    continue
                output_file.write(f"\t{subtopic}\n")
                for outcome in learning_outcomes:
                    output_file.write(f"\t\t- {outcome}\n")

    print(f"Output saved to: {output_txt_path}")


In [10]:
# Processing all 3 PDF files for three different levels and storing extracted  data in 3 different text files
process_pdf('..\\..\\data\\raw-pdf-data\\2024-l1-topics-combined-2.pdf', '..\\..\\data\\extracted-pdf-data_PyPDF2\\PyPDF_RR_2024_l1_combined.txt')
process_pdf('..\\..\\data\\raw-pdf-data\2024-l2-topics-combined-2.pdf', '..\..\data\extracted-pdf-data_PyPDF2\PyPDF_RR_2024_l2_combined.txt')
process_pdf('..\\..\\data\\raw-pdf-data\2024-l3-topics-combined-2.pdf', '..\..\data\extracted-pdf-data_PyPDF2\PyPDF_RR_2024_l3_combined.txt')

Processing ..\..\data\raw-pdf-data\2024-l1-topics-combined-2.pdf
Total number of pages: 27
Output saved to: ..\..\data\extracted-pdf-data_PyPDF2\PyPDF_RR_2024_l1_combined.txt
